# Experiments on Phylo HMMs

In [ ]:
%matplotlib inline
from tree_serialisation import load_tree
from data_simulation import generate_case, rate_sub_HKY, scale_branches_length

import numpy as np
import random
from copy import deepcopy

import matplotlib.pyplot as plt

from felsenstein import pruning
from viterbi_sumproduct import viterbi, sum_product

## First example a toy gene finder

In [26]:
# SIMULATION PARAMETERS
tree_path = "tree.json"
number_of_nucleotids = 100
alphabet = ['A', 'C', 'T', 'G']
alphabetSize = len(alphabet)

nbState = 4
# transition matrix of the toy gene finder
A = np.zeros((nbState, nbState))
A[0, 1] = 1
A[1, 2] = 1
A[2, 3] = 0.33
A[2, 0] = 1 - A[2, 3]
A[3, 3] = 0.33  # 0.9999  # unrealistic ...
A[3, 0] = 1 - A[3, 3]

# state initial probability
b = np.array([0.25, 0.25, 0.26, 0.24])

animalNames = ["dog", "cat", "pig"]#, "cow", "rat", "mouse", "baboon", "human"]

"""[...], such as the higher average rate of substitution and the greater
transition/transversion ratio, in noncoding and third-codon-position sites
than in firstand second- codon-position sites[...]"""

pi = np.zeros((nbState, alphabetSize))
# substitution rates for pi 0 and 1 are between 0 and 0.001
pi[0] = np.random.rand(alphabetSize) * 0.001
pi[1] = np.random.rand(alphabetSize) * 0.001
# but between 0 and 0.01 for pi 2 and 3
pi[2] = np.random.rand(alphabetSize) * 0.01
pi[3] = np.random.rand(alphabetSize) * 0.01
pi /= pi.sum(axis=1)[:, None]

# translation/transversion rate
kappa = np.array([2.3, 2.7, 4.3, 5.4])

In [44]:
# load the phylogenetic model from JSON
tree = load_tree(tree_path)

def sub_tree(tree, number_of_species):
    """
    Prune the tree to keep only number_of_species species
    Args :
           - tree (dict)
           - number_of_species (int)
    """
    tree_cp = deepcopy(tree)
    def treat_node(node):
        childs = tree[node]  # important for the recursion
        if childs:
            for child in childs:
                new_child = child["node"]
                treat_node(new_child)

            for child in childs:
                new_child = child["node"]
                if new_child > number_of_species and not tree_cp[new_child]:
                    # we drop the child
                    tree_cp[node].remove(child)
                
    treat_node(max(tree.keys()))
    # discard "dead leaves"
    for key in tree.keys():
        if key > number_of_species and not tree_cp[key]:
            tree_cp.pop(key, None)
    # now merge useless intermediary node ie with only one son which is not a leave
    node = max(tree.keys())
    while(len(tree_cp[node])== 1 and tree_cp[tree_cp[node][0]["node"]]):
        child = tree_cp[node][0]["node"]
        tree_cp[child][0]["branch"] += tree_cp[node][0]["branch"]
        tree_cp.pop(node, None)
        node = child
    
    return tree_cp
tree = sub_tree(tree, n_species)
print(tree)
trees = []

for j in range(nbState):
    trees.append(scale_branches_length(tree, scale=random.random()))



strands, states = generate_case(A, b, pi, kappa,
                                trees, number_of_nucleotids)

{1: [], 2: [], 3: [], 9: [{'node': 1, 'branch': 0.93}, {'node': 2, 'branch': 0.75}], 10: [{'node': 3, 'branch': 1.02}], 13: [{'node': 9, 'branch': 0.73}, {'node': 10, 'branch': 0.3}]}


In [46]:
# Transform strands from ints to strings
str_strands = list()
for strand in strands:
    str_strand = ""
    for acid_int in strand:
        str_strand = ''.join([str_strand, alphabet[acid_int]])
    str_strands += [str_strand]

In [47]:
# Transform strands in sites
sites = list()
for site_ind in range(number_of_nucleotids):
    sites += [''.join([str_strands[species_ind][site_ind] for species_ind in range(n_species)])]

In [48]:
# Process likelihoods with Felsenstein's algorithm
Qs = rate_sub_HKY(pi, kappa)
likelihoods = np.zeros((nbState, number_of_nucleotids))
for state in range(nbState):
    tree = trees[state]
    Q = Qs[state]
    p = pi[state]
    for site_ind, site in enumerate(sites):
        likelihoods[state, site_ind] = pruning(Q, p, tree, site)

IndexError: list index out of range

In [31]:
# VITERBI PARAMETERS
S = range(nbState)
state_sequence_viterbi = viterbi(S, A, b, likelihoods)

C:\Users\Pierre\Downloads\MVA\phylo-hmm\viterbi_sumproduct.py:44: RuntimeWarning: divide by zero encountered in log
  alpha_log[:, 0] = np.log(b) + np.log(E[:, 0])
C:\Users\Pierre\Downloads\MVA\phylo-hmm\viterbi_sumproduct.py:49: RuntimeWarning: divide by zero encountered in log
  prob = np.log(A[:, s]) + alpha_log[:, t - 1]
C:\Users\Pierre\Downloads\MVA\phylo-hmm\viterbi_sumproduct.py:55: RuntimeWarning: divide by zero encountered in log
  alpha_log[s, t] = np.log(E[s, t]) + prob


In [15]:
# Precision
np.sum(states == state_sequence_viterbi) / number_of_nucleotids

0.28999999999999998